# FloPy

### MT3D-USGS Example

Demonstrates functionality of the flopy MT3D-USGS module using the 'Crank-Nicolson' example distributed with MT3D-USGS.  

#### Problem description:

* Grid dimensions: 1 Layer, 3 Rows, 650 Columns
* Stress periods: 12
* Units are in seconds and days
* Flow package: UPW
* Stress packages: SFR, GHB, UZF
* Solvers: NWT, GCG

In [ ]:
import sys
import os
import flopy.utils.binaryfile as bf
import flopy
import numpy as np
import math

Set up model discretization

In [ ]:
name = 'CrnkNic'
Lx = 650.0 
Ly = 15
nrow = 3
ncol = 650
nlay = 1

delr = Lx / ncol
delc = Ly / nrow

xmax = ncol * delr
ymax = nrow * delc

X, Y = np.meshgrid(np.linspace(delr / 2, xmax - delr / 2, ncol),    
                   np.linspace(ymax - delc / 2, 0 + delc / 2, nrow))


Instantiate MODFLOW object in flopy

In [ ]:
mf = flopy.modflow.Modflow(modelname, exe_name=exe_name, model_ws=directory, version='mfnwt')

Instantiate output control (oc) package for MODFLOW-NWT

In [ ]:
# Output Control: Create a flopy output control object
oc = flopy.modflow.ModflowOc(mf)                      

Instantiate solver package for MODFLOW-NWT

In [ ]:
# Newton-Rhapson Solver: Create a flopy nwt package object

headtol = 1.0E-4 
fluxtol = 5      
maxiterout = 5000
thickfact = 1E-06
linmeth = 2      
iprnwt = 1       
ibotav = 1       
                                                                                      
nwt = flopy.modflow.ModflowNwt(mf, headtol=headtol, fluxtol=fluxtol, maxiterout=maxiterout,       
                               thickfact=thickfact, linmeth=linmeth, iprnwt=iprnwt, ibotav=ibotav,
                               options='SIMPLE')                                                  


Instantiate discretization (DIS) package for MODFLOW-NWT

In [ ]:
# The equations for calculating the ground elevation in the 1 Layer CrnkNic model.  
# Although Y isn't used, keeping it here for symetry
def topElev(X, Y):
    return 100. - (math.ceil(X)-1) * 0.03

grndElev = topElev(X, Y)
bedRockElev = GrndElev - 3.

Steady = [False, False, False]                                                                  
nstp = [1, 1, 1]                                                                        
tsmult = [1., 1., 1.]                                                                        
perlen = [30600, 10800, 45000]                                                                    
                                                                                       
# Create the discretization object
# itmuni = 1 (seconds); lenuni = 2 (meters)
dis = flopy.modflow.ModflowDis(mf, nlay, nrow, ncol, nper=3, delr=delr, delc=delc,     
                               top=grndElev, botm=bedRockElev, laycbd=0, itmuni=1, lenuni=2,
                               steady=Steady, nstp=nstp, tsmult=tsmult, perlen=perlen) 

Instantiate upstream weighting (UPW) flow package for MODFLOW-NWT

In [ ]:
# UPW parameters 
# UPW must be instantiated after DIS.  Otherwise, during the mf.write_input() procedures,   
# flopy will crash.

laytyp = 1   
layavg = 2   
chani = 1.0  
layvka = 1   
iphdry = 0   
hk = 0.1     
hani = 1     
vka = 1.     
ss = 0.000001
sy = 0.20
                                                                                            
upw = flopy.modflow.ModflowUpw(mf, laytyp=laytyp, layavg=layavg, chani=chani, layvka=layvka,
                               ipakcb=53, hdry=hdry, iphdry=iphdry, hk=hk, hani=hani,       
                               vka=vka, ss=ss, sy=sy)                                       


Instantiate basic (BAS or BA6) package for MODFLOW-NWT

In [ ]:
# Create a flopy basic package object                                               
def calc_strtElev(X, Y):
    return 99.5 - (math.ceil(X)-1) * 0.0001

ibound = np.ones((nlay, nrow, ncol))
ibound[:,0,:] *= -1
ibound[:,2,:] *= -1
hdry = -888  
strtElev = calc_strtElev(X, Y)

bas = flopy.modflow.ModflowBas(mf, ibound=ibound, hnoflo=hdry, strt=strtElev)       


In [ ]:
Instantiate streamflow routing (SFR2) package for MODFLOW-NWT

In [ ]:
# Streamflow Routing Package: Try and set up with minimal options in use
# 9 11 IFACE # Data Set 1: ISTCB1  ISTCB2

nstrm = ncol
nss = 6
const = 1.0
dleak = 0.0001
istcb1 = -10
istcb2 = 11
isfropt = 1
# nstrail = 10
# isuzn = 1

nsfrsets = 30
irtflg = 1
numtim = 2
weight = 0.75
flwtol = 0.0001
segment_data = None
channel_geometry_data = None
channel_flow_data = None
dataset_5 = None
reachinput = False

# The next couple of lines set up the reach_data for the 30x100 hypothetical model.
# Will need to adjust the row based on which grid discretization we're doing.
# Ensure that the stream goes down one of the middle rows of the model.

# Determine the middle row and store in rBot (account for 0-base)
rBot = nrow - 1

s1 = u"k,i,j,iseg,ireach,rchlen,iface\n"
for y in xrange(ncol):
    #    layer +    row    +        col   +  iseg +   irch      +     rchlen     +    iface
    s1 += '0,' + str(rBot) + ',' + str(y) + ',1,' + str(y+1) + ',' + str(delr) + ',' + u'0\n'

s1 = StringIO(s1)
reach_data = np.genfromtxt(s1, delimiter=',',names=True, dtype=[('k', '<f8'), ('i', '<f8'), ('j', '<f8'), ('iseg', '<f8'), ('ireach', '<f8'), ('rchlen', '<f8')])

# Will need to adjust the elevations of the upper and lower ends of the segment based on topElev array
# In Cara's original model, these two items appaer to be calculated as 5 m below ground surface (5 m of stream incision)
s2 = StringIO(u"nseg,icalc,outseg,iupseg,nstrpts,   flow,runoff,etsw,pptsw,roughch,roughbk,cdpth,fdpth,awdth,bwdth,hcond1,thickm1,elevup,width1,depth1,hcond2,thickm2,elevdn,width2,depth2\n\
                   1,    1,     0,     0,      0,25000.0,   0.0, 0.0,  0.0,  0.020,    0.0,  0.0,  0.0,  0.0,  0.0,   1.0,    1.0, 254.9,  20.0,   0.0,   1.0,    1.0,225.15,  20.0,   0.0")
segment_data = np.genfromtxt(s2, delimiter=',',names=True)
# Adjust the elevations of the upstream and downstream ends of the segment
ELEVUP = segment_data['elevup'] = topElev[rBot,0] - 10.
ELEVDN = segment_data['elevdn'] = topElev[rBot,ncol-1] - 10.
# Be sure to convert segment_data to a dictionary keyed on stress period.
segment_data = np.atleast_1d(segment_data)
segment_data = {0: segment_data,
                1: segment_data}

# There are 2 stress periods
dataset_5 = {0: [nss, 0, 0],
             1: [nss, 0, 0]}

channel_flow_data = {0: {1:[[ 1.0E+00, 1.0E+00, ELEVUP, 2.0E+01],    #Data set 6b: HCOND1 THICKM1 ELEVUP WIDTH1
                            [ 1.0E+00, 1.0E+00, ELEVDN, 2.0E+01]]},  #Data set 6c: HCOND2 THICKM2 ELEVDN WIDTH2
                     1: {2:[[ 1.0E+00, 1.0E+00, ELEVUP, 2.0E+01],    #Data set 6b: HCOND1 THICKM1 ELEVUP WIDTH1
                            [ 1.0E+00, 1.0E+00, ELEVDN, 2.0E+01]]},} #Data set 6c: HCOND2 THICKM2 ELEVDN WIDTH2

sfr = flopy.modflow.ModflowSfr2(mf, nstrm=nstrm, nss=nss, const=const, dleak=dleak,
                                istcb1=53, istcb2=0, reach_data=reach_data, dataset_5=dataset_5,
                                segment_data=segment_data, channel_flow_data=channel_flow_data)